In [15]:
# CS180 (CS280A): Project 1 starter Python code

# these are just some suggested libraries
# instead of scikit-image you could use matplotlib and opencv to read, write, and display images

import numpy as np
import skimage as sk
import skimage.io as skio

#constants
method = 'l2'

# L2-Norm/Euclidean Distance
def l2_norm(image1, image2):
    return np.sqrt(np.sum(np.sum((image1-image2)**2)))

# Normalized Cross Correlation
def ncc(image1, image2):
    return np.sum(image1 * image2) / (np.linalg.norm(image1) * np.linalg.norm(image2))
    
#Match image1 onto image2
def align(image1, image2, method='l2', displacement_range=15):
    cur_best_diff = None
    cur_best_disp = None

    im1s = sobel(image1)
    im2s = sobel(image2)
    
    #fixed range of 15 pixels
    
    search_range = range(-displacement_range, displacement_range+1)

    for y in search_range:
        for x in search_range:
            rolled_image = np.roll(im1s, (x,y), axis=(0,1))
            if (method == 'l2'):
                metric_result = -l2_norm(rolled_image, im2s)
            if (method == 'ncc'):
                metric_result = ncc(rolled_image, im2s)
    
            if cur_best_diff == None or metric_result > cur_best_diff:
                cur_best_diff = metric_result
                cur_best_disp = (x, y)

    aligned = np.roll(image1, cur_best_disp, axis=(0,1))
    return aligned, np.array(cur_best_disp)

# multi-pass pyramid align
def pyramid(image1, image2, method='ncc', displacement_range=15, pyramid_depth=5):
    if (len(image1) < 100 or pyramid_depth == 0):
        return align(image1, image2, method, displacement_range)
    
    pyramid_image, cur_best_disp = pyramid(sk.transform.rescale(image1, 0.5), sk.transform.rescale(image2, 0.5), method, displacement_range, pyramid_depth=pyramid_depth-1)
    cur_best_disp *=2
    pyramid_image, scaled_shift = align(np.roll(image1, tuple(cur_best_disp), axis=(0,1)), image2, method, displacement_range)
    cur_best_disp = cur_best_disp + scaled_shift
    return pyramid_image, cur_best_disp

def align_helper(image1, image2, method='ncc', displacement_range=15, multi_pass=True):
    if (multi_pass):
        return pyramid(image1, image2, method, 5) # lower displacement for pyramid
    return align(image1, image2, method, displacement_range)

# sobel edge finding
# https://homepages.inf.ed.ac.uk/rbf/HIPR2/sobel.htm
Gx = np.array([[1.0, 0.0, -1.0], [2.0, 0.0, -2.0], [1.0, 0.0, -1.0]])
Gy = np.array([[1.0, 2.0, 1.0], [0.0, 0.0, 0.0], [-1.0, -2.0, -1.0]])
def sobel(image):
    height, width = np.shape(image)
    output = np.zeros(shape=[width, height])

    # must -2 in ranges as our patch is 3x3
    for y in range(height - 2):
        for x in range(width - 2):
            sliced = image[y:y+3, x:x+3]
            dx = np.sum(np.multiply(Gx, sliced))
            dy = np.sum(np.multiply(Gy, sliced))
            output[y+1, x+1] = np.sqrt(dx**2 + dy**2)
    return output

# name of the input file
imname = './cs180 proj1 data/cathedral.jpg'

# read in the image
im = skio.imread(imname)

# convert to double (might want to do this later on to save memory)
im = sk.img_as_float(im)

# compute the height of each part (just 1/3 of total)
height = np.floor(im.shape[0] / 3.0).astype(int)

# separate color channels
b = im[:height]
g = im[height: 2*height]
r = im[2*height: 3*height]

# align the images
# functions that might be useful for aligning the images include:
# np.roll, np.sum, sk.transform.rescale (for multiscale)

ag, ag_disp = align_helper(g, b, method, multi_pass= False)
ar, ar_disp = align_helper(r, b, method, multi_pass= False)
# create a color image
im_out = np.dstack([ar, ag, b])

# save the image
fname = 'out_fname.jpg'
#skio.imsave(fname, im_out)

# display the image
skio.imshow(im_out)
skio.show()

ValueError: operands could not be broadcast together with shapes (3,3) (2,3) 